In [1]:
import datasets
import json
import os
import numpy as np
import collections
import re
import string

In [2]:
# Change these variables to perform evalution for you task
quiz_path = os.path.abspath(os.getcwd()).split('gpt3_evaluation_scripts')[0] + 'generated_data_gpt3/experiment_6/generated_quiz.json'
answer_path = os.path.abspath(os.getcwd()).split('gpt3_evaluation_scripts')[0] + 'generated_data_gpt3/experiment_6/generated_answers_2.json'

In [3]:
# Get quizzes
quizzes = []
with open(quiz_path) as f:
    generated_quizzes = json.load(f)

for key in generated_quizzes:
    quizzes.append(generated_quizzes[key])

# Get questions
answers = []
with open(answer_path) as f:
    generated_answers = json.load(f)

for key in generated_answers:
    answers.append(generated_answers[key])

In [4]:
# Put the predictions and gold references in lists and calculate exact match and f1 score

predictions = []
gold_references = []
predictions_list = []
gold_references_list = []

for i in range(len(quizzes)):
    
    clean_completion = quizzes[i].split('True answer: ')[1].split("\nFalse answer")[0]
    
    generated_completion = answers[i]
    clean_generated_completion = generated_completion.split('Answer: ')[1].replace("\n","")
    
    predictions.append(clean_generated_completion)
    gold_references.append(clean_completion)

    predictions_list.append(clean_generated_completion.split(' '))
    gold_references_list.append([clean_completion.split(' ')])

In [5]:
bleu = datasets.load_metric('bleu')
rouge = datasets.load_metric('rouge')

In [6]:
bleu.add_batch(predictions=predictions_list, references=gold_references_list)
rouge.add_batch(predictions=predictions, references=gold_references)

In [7]:
final_bleu = bleu.compute()
final_rouge = rouge.compute()

In [8]:
# Define path of downloaded meteor from https://www.cs.cmu.edu/~alavie/METEOR/
meteor_path = "<PATH_TO_YOUR_DOWNLOADED_METEOR"

# Move results to meteor directory
with open(meteor_path + "predictions.txt", 'w') as f:
    for i in range(len(predictions)):
        f.write(predictions[i] + '\n')
        
with open(meteor_path + "ground_truth.txt", 'w') as f:
    for i in range(len(gold_references)):
        f.write(gold_references[i] + '\n')

# Run the meteor command from the meteor directory and remove result files again   
wd = os.getcwd()
os.chdir(meteor_path)
output = os.popen("java -Xmx2G -jar meteor-*.jar predictions.txt ground_truth.txt -l en -norm").read()
os.remove(meteor_path + "predictions.txt")
os.remove(meteor_path + "ground_truth.txt")
os.chdir(wd)

# Get the score from the output
meteor_score = round(float(output.split("Final score:")[1].strip()) * 100, 2)

In [9]:
# Create result files

with open(os.path.abspath(os.getcwd()).split('gpt3_evaluation_scripts')[0] + 'generated_data_gpt3/experiment_6/' + 'generated_answers_2.txt', 'w') as f:
    for i in range(len(predictions)):
        f.write(predictions[i] + '\n')
        
with open(os.path.abspath(os.getcwd()).split('gpt3_evaluation_scripts')[0] + 'generated_data_gpt3/experiment_6/' + 'generated_answers.txt', 'w') as f:
    for i in range(len(gold_references)):
        f.write(gold_references[i] + '\n')
        
with open(os.path.abspath(os.getcwd()).split('gpt3_evaluation_scripts')[0] + 'generated_data_gpt3/experiment_6/' + 'automatic_evaluation_answers.txt', 'w') as f:
    f.write("BLEU: " + str(round(final_bleu['bleu'] * 100, 2)))
    f.write('\n')
    f.write("ROUGE-L: " + str(round(final_rouge['rougeL'].mid.fmeasure * 100, 2)))
    f.write('\n')
    f.write("METEOR: " + str(meteor_score))